In [2]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from sklearn.model_selection import train_test_split

import pandas as pd
from datasets import load_dataset, Dataset

/Users/jelenalazovic/Desktop/5/IP2/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jelenalazovic/Desktop/5/IP2/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

df = pd.read_csv('srpski.csv', sep='\t')
df = df.drop(columns=['Rbr', 'SR', 'sr/sr', 'Naslov', 'Jezik'])

df['label'] = df['Autor'].astype('category').cat.codes
df = df.drop(columns=['Autor'])
df = df.rename(columns={'Tekst':'text'})

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 112
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 28
    })
})

In [4]:
model_id = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = SetFitModel.from_pretrained(model_id)
def make_model(params=None):
    return SetFitModel.from_pretrained(
        model_id
    )

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [5]:
trainer = SetFitTrainer(
    model_init=make_model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20,
    num_epochs=1,
    column_mapping={'text':"text", "label": "label"}
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [6]:
def hyperparameter_search_function(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16, 32])    }


In [7]:
trainer.train()

Applying column mapping to training dataset
Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 49.95it/s]
***** Running training *****
  Num examples = 4480
  Num epochs = 1
  Total optimization steps = 280
  Total train batch size = 16
Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
metrics = trainer.evaluate()
metrics

Applying column mapping to evaluation dataset
***** Running evaluation *****


{'accuracy': 0.21739130434782608}